In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pymongo as pm
import numpy as np

x = np.linspace(-2, 2, 101)[np.random.RandomState(0).permutation(101)]
y = x ** 3 - 1

### 1.) Insert x and y into the database "tasks", collection "polyfunction" on host "localhost", port "24444", read it out again in sorted order print "x" and plot "y"

In [ ]:
port = 24444
host = 'localhost'
connection = pm.MongoClient(port = port, host = host)

entries = []
for i in range(101):
    entries.append({'x': x[i], 'y': y[i]})
connection['tasks']['polyfunction'].insert_many(entries)

In [ ]:
entries = connection['tasks']['polyfunction'].find({'x': {'$exists' : True}}, projection=['x', 'y']).sort([('x',pm.ASCENDING)])
x = [entry['x'] for entry in entries]
print(x)
entries = connection['tasks']['polyfunction'].find({'x': {'$exists' : True}}, projection=['x', 'y']).sort([('x',pm.ASCENDING)])
y = [entry['y'] for entry in entries]
plt.plot(x,y)
plt.grid()

### 2.) Run the script below which trains a MLP for the 10-way MNIST classification task 

In [ ]:
import tensorflow as tf
from __future__ import division
from tfutils import base, data, optimizer, utils

# delete exp1
connection['mnist']['simple.files'].delete_many({'exp_id' : 'exp1'})

def mnist_model(inputs, train=True, **kwargs):
    # trainable variables randomly initialized
    with tf.variable_scope("mnist"):
        W1 = tf.get_variable('W1', [784,128], tf.float32, tf.random_normal_initializer(stddev=0.1))
        b1 = tf.get_variable('b1', [128], tf.float32, tf.random_normal_initializer(stddev=0.1))
        W2 = tf.get_variable('W2', [128,32], tf.float32, tf.random_normal_initializer(stddev=0.1))
        b2 = tf.get_variable('b2', [32], tf.float32, tf.random_normal_initializer(stddev=0.1))
        W3 = tf.get_variable('W3', [32,10], tf.float32, tf.random_normal_initializer(stddev=0.1))
        b3 = tf.get_variable('b3', [10], tf.float32, tf.random_normal_initializer(stddev=0.1))
        
        # hidden layers
        h1 = tf.nn.sigmoid(tf.matmul(inputs['images'], W1) + b1, name='hidden1')
        h2 = tf.nn.sigmoid(tf.matmul(h1, W2) + b2, name='hidden2')
        # output
        output = tf.matmul(h2, W3) + b3

    return output, {}

params = {}

params['load_params'] = {
    'do_restore': False}

params['save_params'] = {
    'host': 'localhost',
    'port': 24444,
    'dbname': 'mnist',
    'collname': 'simple',
    'exp_id': 'exp1',
    'save_valid_freq': 200,
    'save_filters_freq': 100,
    'cache_filters_freq': 100}

params['train_params'] = {
    'data_params': {'func': data.MNIST,
                    'batch_size': 256,
                    'group': 'train',
                    'n_threads': 1},
    'queue_params': {'queue_type': 'random',
                     'batch_size': 256},
    'num_steps': 100}

params['model_params'] = {
    'func': mnist_model} 

params['learning_rate_params'] = {
    'learning_rate': 0.5,
    'decay_steps': 500,
    'decay_rate': 0.95,
    'staircase': True}

params['optimizer_params'] = {
    'func': optimizer.ClipOptimizer,
    'optimizer_class': tf.train.MomentumOptimizer,
    'momentum': 0.9,
    'clip': True,
}

params['loss_params'] = {
    'targets': ['labels'],
    'loss_per_case_func': tf.nn.sparse_softmax_cross_entropy_with_logits,
    'agg_func': tf.reduce_mean
}

params['skip_check'] = True

base.train_from_params(**params)

### 3.) Now let's load the trained model from the database and test it on the validation set using TFUtils. There are 10,000 examples in the MNIST test set.
### Fill in the blanks marked with EDIT!

In [ ]:
def top1_func(inputs, outputs): 
    """
    Computes the top1 accuracy with tf.nn.in_top_k
    predictions = outputs
    targets = inputs['labels']
    """
    res = {'top1': EDIT}
    return res

def online_agg(agg_res, res, step):
    """
    Appends the value for each key
    """
    if agg_res is None:
        agg_res = {k: [] for k in res}
    for k, v in res.items():
        agg_res[k].append(v)
    return agg_res

def agg_mean(x):
    """
    Takes the mean of the aggregated results x
    """
    return {k: np.mean(v) for k, v in x.items()}

params = {}

params['load_params'] = {
    'host': EDIT,
    'port': EDIT,
    'dbname': EDIT,
    'collname': EDIT,
    'exp_id': EDIT,
    'do_restore': EDIT}

params['validation_params'] = {'valid0': {
    'data_params': {EDIT},
    'queue_params': {EDIT},
    'targets': {EDIT},
    'num_steps': EDIT,
    'agg_func': EDIT,
    'online_agg_func': EDIT,}}

params['model_params'] = {
    'func': EDIT}

params['skip_check'] = True

base.test_from_params(**params)

# Extract record from database
q_val = {'exp_id' : EDIT, 'validation_results' : {'$exists' : True}, 'validates': {'$exists': True}}
val_steps = connection[EDIT]['EDIT.files'].find(EDIT)
top1 = [val_steps[i][EDIT][EDIT][EDIT] 
        for i in range(val_steps.count())]
print(top1)

### 4.) Execute the script below to load the meta data

In [ ]:
#load from tfrecords
import cPickle
import numpy as np
data_path = '/datasets/neural_data/neural_data.pkl'
with open(data_path) as f:
    data = cPickle.load(f)
meta = data['meta']
IT_features = data['IT']
meta.dtype.names

### 5.) Select all files with 'Animals' or 'Cars' with a rotation in the xy-plane of more than 45 degrees and print how many there are:

In [ ]:
### YOUR CODE HERE

### 6.) Select all objects with 'GORILLA' that either have a variation level of 'V0' or a size 's' bigger 1 or both, and print the object names, their sizes 's' and variation levels as tuples:

In [ ]:
### YOUR CODE HERE

### 7.) Perform a 8-way classification on the categories, using 20 splits, a 'svm.LinearSVC' classifier a 'C' of 5e-3 on variation level 'V0' for train and test.
### Fill in the blanks marked with EDIT!

In [ ]:
from dldata.metrics.utils import compute_metric_base
# Definition of classification experiment
category_eval_spec = {
    'npc_train': None,
    'npc_test': 2,
    'num_splits': EDIT,
    'npc_validate': 0,
    'metric_screen': EDIT,
    'metric_labels': None,
    'metric_kwargs': {'model_type': EDIT,
                      'model_kwargs': {EDIT}
                     },
    'labelfunc': EDIT,
    'train_q': {EDIT},
    'test_q': {EDIT},
    'split_by': 'obj'
}
# Execute classification experiment
res = compute_metric_base(IT_features, meta, category_eval_spec)

# Print results
print('Overall accuracy of IT features on 8-way classification task: %.2f%%' % \
      ((np.array(res['result_summary']['accbal']).mean(0) - 0.5) * 2.0 * 100.0))

### 8.) Plot the confusion matrix

In [ ]:
# Confusion matrix
cms = res['result_summary']['cms']).mean(2)
axis_labels = res['result_summary']['labelset']

### YOUR CODE HERE